In [1]:
from datetime import datetime
from collections import defaultdict

mega_data = []
current_minute = None
minute_buffer = {
    "nifty": {},
    "options": defaultdict(lambda: {"CE": {}, "PE": {}})
}

In [2]:
def process_tick(market_data):
    global current_minute, mega_data, minute_buffer, nifty_spot

    now = datetime.now()
    minute = now.replace(second=0, microsecond=0)
    # If new minute, store the old buffer into mega_data
    if current_minute != minute:
        if current_minute is not None:
            mega_data.append({
                "time": current_minute,
                "nifty": minute_buffer["nifty"],
                "options": dict(minute_buffer["options"])
            })
            # Reset minute_buffer
            minute_buffer = {
                "nifty": {},
                "options": defaultdict(lambda: {"CE": {}, "PE": {}})
            }
        current_minute = minute

    # Update live data
    for symbol, info in market_data.items():
        if symbol == "NIFTY":
            update_ohlc(minute_buffer["nifty"], info)
            if "price" in info:
                nifty_spot = info["price"]
        elif symbol.startswith("NIFTY"):
            try:
                strike = symbol[12:-2]
                option_type = symbol[-2:]
                update_ohlc(minute_buffer["options"][strike][option_type], info)
            except:
                continue



def update_ohlc(ohlc, info):
    price = info.get("price")
    if price is not None:
        ohlc["price"] = price  # ✅ Ensure it's recorded
        if "open" not in ohlc:
            ohlc["open"] = ohlc["high"] = ohlc["low"] = ohlc["close"] = price
        else:
            ohlc["high"] = max(ohlc["high"], price)
            ohlc["low"] = min(ohlc["low"], price)
            ohlc["close"] = price

    for key in ["bbPrice", "bsPrice", "dayVolume", "openInterest"]:
        if key in info:
            ohlc[key] = info[key]


In [ ]:
# ============================
# Block 1: WebSocket + Live Option Table
# ============================

import socketio
import nest_asyncio
import asyncio
import threading
from datetime import datetime
from IPython.display import display, DisplayHandle

# Jupyter fix
nest_asyncio.apply()

# Initialize
sio = socketio.AsyncClient()
latest_data = {}
subscribed_symbols = []
display_handle = DisplayHandle()
display_handle.display("")  # create output cell

nifty_spot = None
selected_expiry = "03JUL25"  # or any expiry you want to track
# ============================
# Table Display Function
# ============================
def display_table():
    output = []
    output.append(f"📈 Live NIFTY Option Prices (Last updated: {datetime.now().strftime('%H:%M:%S')})\n")
    output.append("{:<10} {:>8} {:>8} {:>8} {:>8}   {:>8} {:>8} {:>8} {:>8}".format(
        "Strike", "CE", "Bid", "Ask", "Vol", "PE", "Bid", "Ask", "Vol"))
    output.append("-" * 80)

    if nifty_spot is None:
        output.append("⏳ Waiting for NIFTY price...")
        display_handle.update(output)
        return

    nearest_strike = round(nifty_spot / 50) * 50
    strikes = [nearest_strike + i * 50 for i in range(-5, 6)]

    for strike in strikes:
        ce_key = f"NIFTY{selected_expiry}{strike}CE"
        pe_key = f"NIFTY{selected_expiry}{strike}PE"
        ce = latest_data.get(ce_key, {})
        pe = latest_data.get(pe_key, {})

        def fmt(d, key): return str(d.get(key, "--")).rjust(8)

        row = "{:<10} {} {} {} {}   {} {} {} {}".format(
            strike,
            fmt(ce, "price"), fmt(ce, "bbPrice"), fmt(ce, "bsPrice"), fmt(ce, "dayVolume"),
            fmt(pe, "price"), fmt(pe, "bbPrice"), fmt(pe, "bsPrice"), fmt(pe, "dayVolume")
        )
        output.append(row)

    display_handle.update(output)

# ============================
# WebSocket Events
# ============================
@sio.event
async def connect():
    print("✅ Connected to InstaOptions")
    await sio.emit("subscribe", [[None, "NIFTY"], 100])  # 👈 This is essential


@sio.on("ticker:tick")
async def on_tick(data):
    global nifty_spot
    update_data_time()  # Update last data time for watchdog
    for symbol, info in data.get("marketData", {}).items():
        latest_data[symbol] = info
        if symbol == "NIFTY" and "price" in info:
            nifty_spot = info["price"]
    process_tick(data.get("marketData", {}))  # <-- integrate this


@sio.event
async def disconnect():
    print("🔌 Disconnected")

@sio.event
async def connect_error(data):
    print("❌ Connection error:", data)

# ============================
# Start Socket in Background
# ============================
def start_socket_loop():
    async def main():
        async def refresh():
            while True:
                display_table()
                await asyncio.sleep(3)  # refresh every 10 seconds

        refresh_task = asyncio.create_task(refresh())

        await sio.connect(
            'https://www.instaoptions.in',
            transports=['websocket'],
            socketio_path='analytics-ticker/socket.io'
        )
        await sio.wait()
        refresh_task.cancel()

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main())
import time

last_data_time = time.time()

def update_data_time():
    global last_data_time
    last_data_time = time.time()

def start_watchdog():
    async def watchdog_loop():
        while True:
            await asyncio.sleep(10)
            elapsed = time.time() - last_data_time
            if elapsed > 15:
                print("⚠️ No tick updates received in 15s. Retrying subscription...")
                try:
                    await auto_subscribe_nearby_strikes(expiry="03JUL25", range_count=5)
                except Exception as e:
                    print("❌ Retry failed:", e)
                update_data_time()

    loop = asyncio.new_event_loop()
    threading.Thread(target=lambda: loop.run_until_complete(watchdog_loop()), daemon=True).start()
start_watchdog()
threading.Thread(target=start_socket_loop, daemon=True).start()


['📈 Live NIFTY Option Prices (Last updated: 22:22:06)\n',
 'Strike           CE      Bid      Ask      Vol         PE      Bid      Ask      Vol',
 '--------------------------------------------------------------------------------',
 '25200            --       --       --       --         --       --       --       --',
 '25250            --       --       --       --         --       --       --       --',
 '25300            --       --       --       --         --       --       --       --',
 '25350            --       --       --       --         --       --       --       --',
 '25400            --       --       --       --         --       --       --       --',
 '25450         70.45       --       --       --         --       --       --       --',
 '25500            --       --       --       --         --       --       --       --',
 '25550            --       --       --       --         --       --       --       --',
 '25600            --       --       --       --        

✅ Connected to InstaOptions
🔌 Disconnected
✅ Connected to InstaOptions
🔌 Disconnected
✅ Connected to InstaOptions
🔌 Disconnected
✅ Connected to InstaOptions


packet queue is empty, aborting


🔌 Disconnected
✅ Connected to InstaOptions
🔌 Disconnected
✅ Connected to InstaOptions
🔌 Disconnected
✅ Connected to InstaOptions
⚠️ No tick updates received in 15s. Retrying subscription...
✅ NIFTY Spot Price: 25453.4
📩 Subscribed to 23 symbols near NIFTY 25453.4
⚠️ No tick updates received in 15s. Retrying subscription...
✅ NIFTY Spot Price: 25453.4
📩 Subscribed to 23 symbols near NIFTY 25453.4
⚠️ No tick updates received in 15s. Retrying subscription...
✅ NIFTY Spot Price: 25453.4
📩 Subscribed to 23 symbols near NIFTY 25453.4
⚠️ No tick updates received in 15s. Retrying subscription...
✅ NIFTY Spot Price: 25453.4
📩 Subscribed to 23 symbols near NIFTY 25453.4
⚠️ No tick updates received in 15s. Retrying subscription...
✅ NIFTY Spot Price: 25453.4
📩 Subscribed to 23 symbols near NIFTY 25453.4
⚠️ No tick updates received in 15s. Retrying subscription...
✅ NIFTY Spot Price: 25453.4
📩 Subscribed to 23 symbols near NIFTY 25453.4
⚠️ No tick updates received in 15s. Retrying subscription...


In [ ]:
import pandas as pd
import os

def save_mega_data_to_csv(filename=f"DailyOptionData/nifty_option_megadata_{selected_expiry}_{datetime.today().strftime('%Y-%m-%d')}.csv"):
    flat = []
    for entry in mega_data:
        t = entry['time']

        # Add NIFTY row
        row = {'time': t, 'symbol': 'NIFTY', **entry['nifty']}
        flat.append(row)

        # Add each strike + CE/PE row
        for strike, opts in entry['options'].items():
            for opt_type, values in opts.items():
                row = {'time': t, 'symbol': f"{strike}{opt_type}", **values}
                flat.append(row)

    df = pd.DataFrame(flat)

    if os.path.exists(filename):
        try:
            existing = pd.read_csv(filename, parse_dates=['time'])

            # Handle empty or corrupt files
            if existing.empty or 'time' not in existing.columns:
                print("⚠️ Existing CSV is empty or malformed, overwriting.")
                existing = pd.DataFrame(columns=df.columns)

            df = df[~df['time'].isin(existing['time'])]
            df = pd.concat([existing, df], ignore_index=True).drop_duplicates(subset=["time", "symbol"])
        except Exception as e:
            print("⚠️ Failed to read existing CSV, overwriting file:", e)

    df.to_csv(filename, index=False)
    print(f"✅ Data saved to {filename}")
import time
import threading

def periodic_saver(interval_minutes=1):
    while True:
        try:
            save_mega_data_to_csv()
        except Exception as e:
            print("❌ Auto-save failed:", e)
        time.sleep(interval_minutes * 60)

threading.Thread(target=periodic_saver, daemon=True).start()


✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv


✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv
✅ Data saved to DailyOptionData/nifty_option_megadata_10JUL25_2025-07-02.csv

In [5]:
# ============================
# Block 2: Subscribe to strikes
# ============================
async def subscribe_to_strikes(strike_list, expiry="26JUN25"):
    global subscribed_symbols
    symbols = [f"NIFTY{expiry}{strike}CE" for strike in strike_list] + [f"NIFTY{expiry}{strike}PE" for strike in strike_list]
    await sio.emit("subscribe", [[None] , symbols, 100])
    subscribed_symbols = list(set(subscribed_symbols + symbols))  # avoid duplicates
    print(f"📩 Subscribed to: {symbols}")

async def auto_subscribe_nearby_strikes(expiry="03JUL25", range_count=5):
    global subscribed_symbols, selected_expiry
    selected_expiry = expiry  # ✅ set expiry for display

    # ⏳ Wait for NIFTY price to be available
    while "NIFTY" not in latest_data or "price" not in latest_data["NIFTY"]:
        print("⏳ Waiting for NIFTY spot price...")
        await asyncio.sleep(1)

    nifty_spot = latest_data["NIFTY"]["price"]
    print(f"✅ NIFTY Spot Price: {nifty_spot}")

    nearest = round(nifty_spot / 50) * 50
    strike_list = [nearest + i * 50 for i in range(-range_count, range_count + 1)]

    # Subscribe to NIFTY + nearby strikes
    symbols = ["NIFTY"] + \
              [f"NIFTY{expiry}{strike}CE" for strike in strike_list] + \
              [f"NIFTY{expiry}{strike}PE" for strike in strike_list]

    await sio.emit("subscribe", [[None] + symbols, 100])
    subscribed_symbols = list(set(subscribed_symbols + symbols))
    print(f"📩 Subscribed to {len(symbols)} symbols near NIFTY {nifty_spot}")


In [ ]:
await auto_subscribe_nearby_strikes(expiry="10JUL25")

✅ NIFTY Spot Price: 25491.5
📩 Subscribed to 23 symbols near NIFTY 25491.5


<h3>Applying our model</h3>

In [7]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
model = xgb.Booster()
model.load_model("Saved_XGB_models/final_model.ubj")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit([ "BUY", "NO", "STRONG BUY"])


LabelEncoder()

In [8]:
last_predicted_time = None

def predict_from_latest_minute():
    global last_predicted_time 
    if not mega_data:
        print("⏳ Waiting for mega_data to build...")
        return

    latest_minute = mega_data[-1]
    if latest_minute['time'] == last_predicted_time:
        # print("nskjgdnks")
        return  # Already predicted for this minute
    if "nifty" not in latest_minute or "price" not in latest_minute["nifty"]:
        print("⚠️ NIFTY data incomplete for prediction")
        return

    current_price = latest_minute["nifty"]["price"]
    nearest_strike = round(current_price / 50) * 50
    strike_range = [str(nearest_strike + i * 50) for i in range(-3, 4)]

    expiry_dt = datetime(2025, 7, 3, 15, 30)  # You can make this dynamic later

    print(f"\n🧠 Predicting at {latest_minute['time'].strftime('%H:%M:%S')}")
    for strike in strike_range:
        for opt_type in ["CE", "PE"]:
            if strike in latest_minute["options"]:
                try:
                    label = predict_xgb_action(strike, opt_type, expiry_dt, latest_minute)
                    if label in ["BUY", "STRONG BUY"]:
                        print(f"✅ {strike}{opt_type}: {label}")
                except Exception as e:
                    print(f"❌ Error predicting {strike}{opt_type}:", e)
    last_predicted_time= latest_minute['time']


In [9]:
from datetime import datetime
import xgboost as xgb
import numpy as np
def build_feature_sequence(strike, option_type, expiry_dt):
    """
    Builds 15-timestep x 10-feature input for a given strike and option_type (CE/PE).
    expiry_dt: datetime object like datetime(2025, 7, 3, 15, 30)
    Returns: np.array shape (15, 10)
    """
    sequence = []
    for minute_data in mega_data[-15:]:
        nifty = minute_data.get("nifty", {})
        option_data = minute_data.get("options", {}).get(str(strike), {}).get(option_type, {})

        if not nifty or not option_data:
            # Fill with zeros if missing
            sequence.append([0]*10)
            continue

        tte = (expiry_dt - minute_data["time"]).total_seconds() / (60 * 60 * 24)

        features = [
            nifty.get("open", 0),
            nifty.get("close", 0),
            nifty.get("high", 0),
            nifty.get("low", 0),
            1 if option_type == "CE" else -1,
            option_data.get("open", 0),
            option_data.get("close", 0),
            option_data.get("openInterest", 0),
            float(strike),
            tte
        ]
        sequence.append(features)

    return np.array(sequence)
def predict_xgb_action(strike, opt_type, expiry_dt, latest_minute):
    try:
        # collect past 15 minutes of data
        recent = mega_data[-15:]
        if len(recent) < 15:
            raise ValueError("Not enough data for 15 timesteps")

        feature_rows = []
        for minute in recent:
            nifty = minute.get("nifty", {})
            option = minute["options"].get(str(strike), {}).get(opt_type, {})

            time_to_expiry = (expiry_dt - minute['time']).total_seconds() / (60 * 60 * 24)
            row = [
                nifty.get("open", 0),
                nifty.get("close", 0),
                nifty.get("high", 0),
                nifty.get("low", 0),
                1 if opt_type == "CE" else -1,
                option.get("open", 0),
                option.get("close", 0),
                option.get("openInterest", 0),
                float(strike),
                time_to_expiry
            ]
            feature_rows.append(row)

        x_input = np.array(feature_rows).reshape(1, -1)

        # print("🧪 Predicting with X:")
        # print(x_input)

        dmatrix = xgb.DMatrix(x_input)
        label_index = model.predict(dmatrix)[0]
        # print("📊 Raw prediction index:", label_index)
        return le.inverse_transform([int(label_index)])[0]

    except Exception as e:
        print(f"❌ Prediction error for {strike}{opt_type}: {e}")
        return "ERROR"


In [10]:
import threading
import asyncio

def start_prediction_loop():
    async def prediction_runner():
        while True:
            predict_from_latest_minute()
            await asyncio.sleep(10)  # Run every 10 seconds

    loop = asyncio.new_event_loop()
    threading.Thread(target=lambda: loop.run_until_complete(prediction_runner()), daemon=True).start()


In [11]:
start_prediction_loop()

⏳ Waiting for mega_data to build...


⏳ Waiting for mega_data to build...

🧠 Predicting at 10:38:00
❌ Prediction error for 25350CE: Not enough data for 15 timesteps
❌ Prediction error for 25350PE: Not enough data for 15 timesteps
❌ Prediction error for 25400CE: Not enough data for 15 timesteps
❌ Prediction error for 25400PE: Not enough data for 15 timesteps
❌ Prediction error for 25450CE: Not enough data for 15 timesteps
❌ Prediction error for 25450PE: Not enough data for 15 timesteps
❌ Prediction error for 25500CE: Not enough data for 15 timesteps
❌ Prediction error for 25500PE: Not enough data for 15 timesteps
❌ Prediction error for 25550CE: Not enough data for 15 timesteps
❌ Prediction error for 25550PE: Not enough data for 15 timesteps
❌ Prediction error for 25600CE: Not enough data for 15 timesteps
❌ Prediction error for 25600PE: Not enough data for 15 timesteps
❌ Prediction error for 25650CE: Not enough data for 15 timesteps
❌ Prediction error for 25650PE: Not enough data for 15 timesteps

🧠 Predicting at 10:39:00
❌ 